## 인스타그램 크롤링 

### 인스타그램 검색 결과 URL을 만들어서 접속하기
### 로그인해야 검색 가능함으로 로그인 처리할것.

In [4]:
# 로그인하기
# selenium으로 URL 접속하기
# 계정: 
# 패스워드: 
import time
from selenium import webdriver

driver = webdriver.Chrome("C:/ai6/chromedriver.exe")   
driver.set_window_size(1000, 800)

driver.get("https://www.instagram.com")
time.sleep(3)

In [5]:
# login = driver.find_element_by_xpath('')
# login.click()
# time.sleep(2)

# id 
id = driver.find_element_by_css_selector('#react-root > section > main > article > div.rgFsT > div:nth-child(1) > div > form > div:nth-child(2) > div > label > input')
id.send_keys('회원 아이디')

# password
passwd = driver.find_element_by_css_selector('#react-root > section > main > article > div.rgFsT > div:nth-child(1) > div > form > div:nth-child(3) > div > label > input')
passwd.send_keys('회원 패스워드')

# 로그인 
login_btn = driver.find_element_by_css_selector('#react-root > section > main > article > div.rgFsT > div:nth-child(1) > div > form > div:nth-child(4) > button')
login_btn.click()
time.sleep(2)  # 로그인 처리에 2초 예상

In [6]:
# 알림 설정 -> 나중에하기 버튼
login_btn = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm')
login_btn.click()
time.sleep(1)

In [31]:
# 알림 설정 - 나중에 하기 클릭
window_close = driver.find_element_by_xpath('/html/body/div[4]/div/div/div[3]/button[2]')
window_close.click()
time.sleep(2)

In [7]:
# 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url    

In [8]:
word = "제주도맛집"
url = insta_searching(word)
driver.get(url)
time.sleep(2)

In [11]:
# 첫번째 게시글 열기
first = driver.find_element_by_css_selector("div._9AhH0")
first.click()
time.sleep(3)

In [13]:
# 다음 게시글 열기
# right = driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/div/a')
# body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow
right = driver.find_element_by_css_selector ('a._65Bje.coreSpriteRightPaginationArrow')
right.click()
time.sleep(3)

### 첫 번째 게시글 열기 

In [14]:
# HTML에서 첫번째 게시글 찾아 클릭하기 함수
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

### 게시글 정보 가져오기 

In [15]:
# 게시글 정보 가져오기
import re

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ② 본문 내용 가져오기
    try:
        # 첫번째 올린글, 두번째부터 이어지는 댓글
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
        
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    # r'#[^\s#,\\]+': #으로 시작하고
    # #뒤에 연속된 문자로 공백이나 #, \ 기호가 아닌 경우의
    # 문자를 찾아서 list 형태로 tags 변수에 저장
    tags = re.findall(r'#[^\s#,\\]+', content)  
    
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
        
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''

    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data


### 다음 게시글 열기

In [16]:
# 다음 게시글 열기
def move_next(driver):
    right = driver.find_element_by_css_selector ('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

### 여러 게시글 정보 수집하기

In [17]:
# 인스타그램 크롤링
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# ① 크롬 브라우저 열기
# driver = webdriver.Chrome("F:/py_202003/chromedriver.exe")   

# ② 인스타그램 검색페이지 URL 만들기
word = "제주도맛집"    #검색어
# word = "제주도관광"    #검색어
# word = "제주맛집"    #검색어
# word = "제주여행"    #검색어

url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(3)

# ④ 첫 번째 게시글 열기
select_first(driver)

# ⑤ 비어있는 변수(results)만들기
results = [ ]

# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 5      # 크롤링할 게시글 수
for i in range(target):
    data = get_content(driver)    # 게시글 정보 가져오기
    results.append(data)
    move_next(driver)

print(results[:2])

[['매일 매일 방역은 필수 농식품부 선정 코로나 방역 우수식당!!국민이 추천해준 안전식당고집돌우럭!!!더 안전하게 즐길 수 있도록 오늘도 쉼없이 갑니다!! #농림축산식품부 #코로나방역우수식당 #국민추천 #안심맛집 #제주맛집', '2020-07-03', '1,186', '고집돌우럭중문점', ['#농림축산식품부', '#코로나방역우수식당', '#국민추천', '#안심맛집', '#제주맛집']], ['고집돌우럭 오늘도 방역 완료 농식품부가 선정한 전국 코로나방역 우수 식당에 당당히 이름을 올렸습니다!! 제주에서 오직 두곳!! 국민이 추천한 우수방역업소입니다!! 묵묵히 하던일을 계속 하다보니 이런 좋은 결과도 생기네요!! 더 안전한 먹거리로 모실께요!! .우수방역업소 선정을 기념해서 작은 이벤트 진행합니다!! 댓글에 👍 엄지척 이모티콘을 남겨주시면 2분을 선정하여 코로나방역키트를 선물로 쏩니다!! #코로나방역 #안심식당 #농식품부 #방역우수식당 #고집돌우럭 #제주도맛집', '2020-07-03', '1,268', 'gozip_hamdeok', ['#코로나방역', '#안심식당', '#농식품부', '#방역우수식당', '#고집돌우럭', '#제주도맛집']]]


In [20]:
# 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']

# 제주도 맛집 2020 6월 기준
results_df.to_excel('./data/crawling_jejudoMatJip_202006.xlsx', index = False)

# 제주도 맛집:jejudoMatJip.xlsx 
# results_df.to_excel('./data/jejudoMatJip.xlsx', index = False)

# 제주도 관광:jejudoGwanGwang.xlsx 
# results_df.to_excel('./data/jejudoGwanGwang.xlsx', index = False)

# 제주 맛집: jejuMatJip.xlsx
# results_df.to_excel('./data/jejuMatJip.xlsx', index = False)

# 제주 여행: jejuYeoHang.xlsx
# results_df.to_excel('./data/jejuYeoHang.xlsx', index = False)

###  여러 엑셀 파일의 중복을 제거한 후 통합 저장


In [27]:
import pandas as pd

# 여러 개의 저장파일 통합하기
# 제주도 맛집:jejudoMatJip.xlsx 
# 제주도 관광:jejudoGwanGwang.xlsx 
# 제주 맛집: jejuMatJip.xlsx
# 제주 여행: jejuYeoHang.xlsx

jeju_insta_df = pd.DataFrame( [ ] )

folder = './data/'
f_list = ['jejudoMatJip.xlsx', 'jejudoGwanGwang.xlsx', 'jejuMatJip.xlsx', 'jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    # 하나의 DataFrame 으로 결합
    jeju_insta_df = jeju_insta_df.append(temp) 

# 컬럼명 적용
jeju_insta_df.columns =['content','data','like','place','tags']


In [28]:
# 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)

# 최종 데이터 파일로 저장
jeju_insta_df.to_excel('./data/crawling_raw.xlsx', index = False)